In [ ]:
import Pkg
Pkg.activate(".")

pkgs = [
"ProgressMeter",
"uCSV",
"DataFrames",
"GenomicAnnotations",
"StatsBase",
"Statistics",
"FASTX",
"BioAlignments"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

# Pkg.develop(path="../../..")
Pkg.develop(path="$(homedir())/workspace/Mycelia")
# import Mycelia
import Mycelia

In [ ]:
# run(`pip install csvkit`)

In [ ]:
# Genus = Enterovirus
# Enterovirus A
# Enterovirus B
# Enterovirus C
# Enterovirus D
# Rhinovirus A
# Rhinovirus B

# Genus = mastadenovirus
# Human mastodenovirus D



# Influenza B virus
# Respiratory syncytial virus
# SARS related coronavirus
# Epstein-Barr virus (HHV-4)
# B19 virus
# Human herpevirus 6B(HHV-6B)
# Merkel cell polyomavirus(MCPyV)

In [ ]:
# viral_metadata_file = "../metadata/VMR_MSL38_v1 - VMR MSL38 v1.tsv"
# viral_metadata_table = DataFrames.DataFrame(uCSV.read(viral_metadata_file, header=1, delim='\t')...)

In [ ]:
genbank_file = "../NC_002058.gb"
genbank_record = first(GenomicAnnotations.readgbk(genbank_file))

In [ ]:
gene_lengths = Int[]
for gene in genbank_record.genes
    if !ismissing(gene.protein_id)
        l = length(GenomicAnnotations.locus(gene).position)
        # @show gene
        push!(gene_lengths, l)
    end
end
threshold = Statistics.mean(gene_lengths) + 1Statistics.std(gene_lengths)

In [ ]:
sum(filter(x -> x >= threshold, gene_lengths))

In [ ]:
fasta_file = replace(genbank_file, ".gb" => ".fna")
open(fasta_file, "w") do io
    fasta_writer = FASTX.FASTA.Writer(io)
    for gene in genbank_record.genes
        if !ismissing(gene.protein_id)
            l = length(GenomicAnnotations.locus(gene).position)
            if l >= threshold
                record = FASTX.FASTA.Record(gene.product, GenomicAnnotations.sequence(gene))
                # @show record
                write(fasta_writer, record)
            end
        end
    end
    close(fasta_writer)
end

In [ ]:
enteroviruses = collect(Mycelia.open_fastx("../enteroviruses.fna"))

In [ ]:
total_length = 0
for x in filter(x -> occursin(".fna", x), readdir("../sequences", join=true))
    for record in Mycelia.open_fastx(x)
        total_length += length(FASTX.sequence(record))
    end
end
total_length

In [ ]:
# a = 3
# b = 6
# aln = BioAlignments.pairalign(
#     BioAlignments.LevenshteinDistance(),
#     FASTX.sequence(enteroviruses[a]),
#     FASTX.sequence(enteroviruses[b])
# )
# length(BioAlignments.sequence(aln))